In [1]:


from ConvergenceAnalysis import *


In [2]:
fd_beta_data
data_path=Path('/home/ahurta92/data')
# rename System to Molecule
fd_beta_data.rename(columns={'System': 'Molecule'}, inplace=True)
mols = fd_beta_data.Molecule.unique().tolist()
# remove KF 
mols.remove('KF')
mols.remove('LiBr')


In [3]:
fd_beta_data

In [4]:
# query molecule LiH 
fd_beta_data.query('Molecule == "LiH"')

In [5]:
from DataAnalysisClass import *
beta_convergence_path=data_path/ "october_beta_absolute_kain"
paper_path = Path('beta_convergence_figures')
# mkdir for the paper path
paper_path.mkdir(exist_ok=True)
print(beta_convergence_path)



In [6]:

single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized + double + double_polarized


In [7]:
low_q_data = QuadraticDatabase(mols, all_basis_sets, 'hf', 'dipole', [0], beta_convergence_path
                               .joinpath('low-low'), True)
high_q_data = QuadraticDatabase(mols, all_basis_sets, 'hf', 'dipole', [0],
                                beta_convergence_path
                                .joinpath('high-high'), True)
# save data

In [31]:
low_q_data.save_dfs()
high_q_data.save_dfs()


In [14]:
low_q_data.beta_hrs_df.query('basis=="aug-cc-pVDZ" & Afreq==0.0')

In [9]:
sns plotting context to talk
sns.set_context('talk')

In [10]:

# get the percent error relative to the FD datal
beta_data = low_q_data.q_df.copy()
beta_data = high_q_data.q_df.copy()

low_z_data = create_z_beta_data(low_q_data.q_df)
# add column called MRA-LOW
low_z_data['Protocol'] = 'Medium'
high_z_data = create_z_beta_data(high_q_data.q_df)
# add column called MRA-HIGH
high_z_data['Protocol'] = 'High'
# concatenate the two dataframes
z_data = pd.concat([low_z_data, high_z_data], axis=0)

z_data


In [11]:

beta_z_plot = compare_z_beta_to_basis_set(z_data, 'Absolute Percent Error')
# save the plot
beta_z_plot.savefig(paper_path.joinpath('beta_z_plot_2.svg'), dpi=600)


In [12]:
z_data.query('basis=="MRA"').sort_values('molecule')

In [13]:



low_data = PolarizabilityData(mols, 'hf', 'dipole', all_basis_sets,
                              october_absolute_kain_path.joinpath
                              ('low-low'), overwrite=False)
high_data = PolarizabilityData(mols, 'hf', 'dipole', all_basis_sets, october_absolute_kain_path
                               .joinpath('high-high'), overwrite=False)

low_data.save_dfs()
high_data.save_dfs()
low_data.polar_data.query('omega==0')
low_data.iso_data.query('omega==0')


In [14]:
low_z_data = create_z_data(low_data, 'two')
# create Protocol column
low_z_data['Protocol'] = 'Medium'
high_z_data = create_z_data(high_data, 'two')
high_z_data['Protocol'] = 'High'
z_data = pd.concat([low_z_data, high_z_data])
z_data
# remove HeNe
z_data = z_data.query('molecule!="HeNe"')



In [15]:
z_abs_error_plot = compare_z_to_basis_set(z_data, y='Absolute Percent Error')
z_abs_error_plot.savefig(paper_path.joinpath('z_abs_error_plot_2.svg'), dpi=600)


In [15]:
low_data.iso_data.query('omega==0')
# grab the basis set data only 
basis_data = low_data.iso_data.query('omega==0 and basis!="MRA"').copy()
# add a column for the MRA data for each molecule by mapping based on molecule
basis_data['alpha(MRA)'] = basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                              'basis=="MRA"').set_index
                                                      ('molecule')['alpha'].to_dict())
basis_data['gamma(MRA)'] = basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                              'basis=="MRA"').set_index
                                                      ('molecule')['gamma'].to_dict())
#basis_data
# compute the percent error in alpha and gamma
basis_data['alpha error'] = 100 * (basis_data['alpha'] - basis_data['alpha(MRA)']) / basis_data[
    'alpha(MRA)']
basis_data['gamma error'] = 100 * (basis_data['gamma'] - basis_data['gamma(MRA)']) / basis_data[
    'gamma(MRA)']


In [16]:
basis_data = make_detailed_df(basis_data)
basis_data


In [18]:
low_data.iso_data.query('omega==0')
# grab the basis set data only 
basis_data = low_data.iso_data.query('omega==0 and basis!="MRA"').copy()
# add a column for the MRA data for each molecule by mapping based on molecule
basis_data['alpha(MRA)'] = basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                              'basis=="MRA"').set_index
                                                      ('molecule')['alpha'].to_dict())
basis_data['gamma(MRA)'] = basis_data['molecule'].map(low_data.iso_data.query('omega==0 and '
                                                                              'basis=="MRA"').set_index
                                                      ('molecule')['gamma'].to_dict())
#basis_data
# compute the percent error in alpha and gamma
basis_data['alpha error'] = 100 * (basis_data['alpha'] - basis_data['alpha(MRA)']) / basis_data[
    'alpha(MRA)']
basis_data['gamma error'] = 100 * (basis_data['gamma'] - basis_data['gamma(MRA)']) / basis_data[
    'gamma(MRA)']



In [24]:
basis_data = high_q_data.beta_hrs_df.query('basis!="MRA" & Afreq==0.0').copy()
basis_data['beta(MRA)'] = basis_data['molecule'].map(low_q_data.beta_hrs_df.query('basis=="MRA" & '
                                                                                  'Afreq==0.0')
                                                     .set_index('molecule')['Beta'].to_dict())
basis_data['betaE'] = 100 * (basis_data['Beta'] - basis_data['beta(MRA)']) / basis_data['beta(MRA)']
basis_data = make_detailed_df(basis_data)
basis_data

In [28]:
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

aspect_ratio = .5
Type_map = {"aug-cc-pVnZ": "sn", "aug-cc-pCVnZ": "sCn", "d-aug-cc-pVnZ": "dn",
            "d-aug-cc-pCVnZ": "dCn"}
vlevel = ['D', 'T', 'Q', '5']

plot_data = basis_data.query('valence.isin(@vlevel) and Afreq==0').copy()
with sns.plotting_context('paper', font_scale=1.00):
    g = sns.catplot(col='valence', x='Type', y='betaE', hue='mol_system',
                    data=plot_data, kind='strip', dodge=True, jitter=True,
                    s=15, palette='colorblind', height=4, aspect=aspect_ratio, sharey=True,
                    alpha=0.5,
                    sharex=False)
    # adjust the title to read D=$\zeta$D, T=$\zeta$T, etc.
    g.set_titles('{col_name}Z')
    g.set_xlabels('')

    g.set_xticklabels([Type_map[label.get_text()] for label in g.axes.flat[0].get_xticklabels()])
    v = 0
    for ax in g.axes.flat:
        # draw horizontal line at .05%
        # get the valence level from the title
        c = vlevel[v]
        v += 1
        print(c)
        xl = ['aug', 'aug+core', 'd-aug', 'd-aug+core']
        xl = ['s-n', 's-Cn', 'd-n', 'd-Cn']
        xl = ['sn', 'sCn', 'dn', 'dCn']
        ax.set_xticklabels(xl)

        ax.set_yscale('symlog', linthresh=1e-1)
        ax.axhline(y=.02, linestyle='--', color='orange')
        ax.axhline(y=-.02, linestyle='--', color='orange')
        # for the symlog scale to regular notation with no decimal places for the y axis
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.yaxis.set_major_formatter(FormatStrFormatter('%2g'))

        # make the y limits symmetrical for each plot using the max absolute value
        max_lim = max(abs(ax.get_ylim()[0]), abs(ax.get_ylim()[1]))
        ax.set_ylim(-30, 30)

        ax.set_ylabel(r'$\beta_{HRS}$ Error (%)')
        # rotate the xlabels
        for label in ax.get_xticklabels():
            label.set_rotation(0)
            #label.set_horizontalalignment('right')
        # create vertical lines to separate basis sets
        for i in range(1, len(plot_data.Type.unique())):
            ax.axvline(i - .5, linestyle='-.', color='k', linewidth=.5, alpha=.8)
        for i in range(1, 5):
            ax.axvspan((i - 1) - .5, i - .5, 1e-3, facecolor=pal[i - 1], alpha=0.1)
        # remove the legend title


In [30]:
import glob

beta_path = Path('/mnt/data/madness_data/august_no_symmetry')
beta_mol = '/mnt/data/madness_data/august_no_symmetry/molecules/*.mol'

# glob for .mol files in august molecules directory
mols = glob.glob(beta_mol)
# remove .mol extension
mols = [mol.split('/')[-1].split('.')[0] for mol in mols]
print(mols)


In [32]:

beta_path = Path('/mnt/data/madness_data/august_no_symmetry')
quad_data = QuadraticDatabase(mols, all_basis_sets, 'hf', 'dipole', [0], beta_path, False)
quad_data.save_dfs()
# save data
